In [1]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# fine tune model

In [2]:
#yolo settings datasets_dir='/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2'

In [3]:
base_model = "../models/yolo11s-seg.pt"
dataset_path = "/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/data.yaml"

In [4]:
# check if the dataset exists
if not os.path.exists(dataset_path):
    print("Dataset not found")
    exit()

# check if model exists
if not os.path.exists(base_model):
    print("Model not found")
    exit()

In [5]:
# Load a model
model = YOLO(base_model)

# Train the model
train_results = model.train(
    data=dataset_path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

Ultralytics 8.3.78 🚀 Python-3.12.7 torch-2.5.1 CPU (Apple M2 Pro)
engine/trainer: task=segment, mode=train, model=../models/yolo11s-seg.pt, data=/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/train/labels... 366 images, 1 backgrounds, 0 corrupt: 100%|██████████| 366/366 [00:00<00:00, 1056.37it/s]

train: New cache created: /Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/train/labels.cache



val: Scanning /Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/valid/labels... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<00:00, 1844.54it/s]

val: WARNING ⚠️ /Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/valid/images/corcieux_lumix_E022B_le_1_jpg.rf.d7511483b2ee8b11c087871ee250ccf3.jpg: 1 duplicate labels removed
val: New cache created: /Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/yolov12_t-tfzxm_logsv2/valid/labels.cache


...

      1/100         0G     0.5016     0.9778      1.706       1.06         37        640: 100%|██████████| 23/23 [07:17<00:00, 19.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:44<00:00, 11.08s/it]

                   all        108        111      0.245      0.748      0.284      0.205      0.245      0.748      0.284      0.257



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      2/100         0G     0.5402     0.6672     0.7582      1.058         44        640:  83%|████████▎ | 19/23 [06:24<01:20, 20.24s/it]


KeyboardInterrupt: 

In [ ]:
# Evaluate model performance on the validation set
metrics = model.val()

In [ ]:
# Export the model as pytorch model
path = model.export()


# results

## Plot segmented masks

In [ ]:
results = model("images/sofa.jpg")

In [ ]:
for result in results:
    height, width = result.orig_img.shape[:2]
    background = np.ones((height, width, 3), dtype=np.uint8) * 255

    masks = result.masks.xy
    for mask in masks:
        mask = mask.astype(int)
        cv2.drawContours(background, [mask], -1, (0, 255, 0), thickness=cv2.FILLED)

    plt.imshow(background)
    plt.title('Segmented objects')
    plt.axis('off')
    plt.show()

    plt.imsave('segmented_objects.jpg', background)

## Plot segmentation mask with original colors

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

for result in results:
    height, width = result.orig_img.shape[:2]
    background = np.ones((height, width, 3), dtype=np.uint8) * 255
    masks = result.masks.xy
    orig_img = result.orig_img

    for mask in masks:
        mask = mask.astype(int)
        mask_img = np.zeros_like(orig_img)

        cv2.fillPoly(mask_img, [mask], (255, 255, 255))
        masked_object = cv2.bitwise_and(orig_img, mask_img)
        background[mask_img == 255] = masked_object[mask_img == 255]

    background_rgb = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)

    plt.imshow(background_rgb)
    plt.title('Segmented objects')
    plt.axis('off')
    plt.show()

    cv2.imwrite('segmented_objects.jpg', background)